# <center>АНАЛИЗ ЗВУКА И ГОЛОСА</center>

**Преподаватель**: Рыбин Сергей Витальевич

**Группа**: 6304

**Студент**: Белоусов Евгений Олегович

## <center>Классификация произвольных звуков</center>
### <center>(Kaggle's Freesound General-Purpose Audio Tagging Challenge)</center>

*Необоходимый результат: больше 0.76 (10 баллов), не меньше 0.7 (8 баллов), не меньше 0.33 (6 баллов)*

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil

import IPython
import numpy as np
import pandas as pd
import librosa

from sklearn.model_selection import StratifiedKFold

from tensorflow.keras import losses, models, optimizers
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, TensorBoard)

from tensorflow.keras.layers import (Input, Dense, Convolution2D, BatchNormalization,
                                     Flatten, MaxPool2D, Activation)
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import backend as K

In [2]:
np.random.seed(1001)

In [3]:
# Загружаем датасет

train = pd.read_csv("./SVA/train.csv")
test = pd.read_csv("./SVA/sample_submission.csv")

In [4]:
train.head()

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1


In [5]:
test.head()

,fname,label
0,00063640.wav,Laughter Hi-Hat Flute
1,0013a1db.wav,Laughter Hi-Hat Flute
2,002bb878.wav,Laughter Hi-Hat Flute
3,002d392d.wav,Laughter Hi-Hat Flute
4,00326aa9.wav,Laughter Hi-Hat Flute


In [6]:
# Базовая информация о датасете

print("Количество тренировочных сэмплов: ", train.shape[0])
print("Количество классов тренировочных сэмплов: ", len(train.label.unique()))

Количество тренировочных сэмплов:  9473
Количество классов тренировочных сэмплов:  41


In [7]:
# Параметры конфигурации для будущей модели нейросети

class Config(object):
    def __init__(self,
                 sampling_rate=16000, audio_duration=2, n_classes=41,
                 use_mfcc=False, n_folds=10, learning_rate=0.0001, 
                 max_epochs=50, n_mfcc=20):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_mfcc = use_mfcc
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs

        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length / 512)), 1)
        else:
            self.dim = (self.audio_length, 1)

In [8]:
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index("fname", inplace=True)
test.set_index("fname", inplace=True)
train["label_idx"] = train.label.apply(lambda x: label_idx[x])

In [9]:
# Модель свёрточной нейросети

def get_2d_conv_model(config):
    
    nclass = config.n_classes
    
    inp = Input(shape=(config.dim[0],config.dim[1],1))
    
    x = Convolution2D(32, (4,10), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    
    x = Convolution2D(32, (4,10), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    
    x = Convolution2D(32, (4,10), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    out = Dense(nclass, activation=softmax)(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(config.learning_rate)

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    return model

In [11]:
# Первичная обработка данных

def prepare_data(df, config, data_dir):
    X = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    input_length = config.audio_length
    for i, fname in enumerate(df.index):
        # Загрузка аудиофайла
        print(i)
        print(fname)
        file_path = data_dir + fname
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate, res_type="kaiser_fast")

        # Обрезка/приведение длительности аудиофайла к указанной в параметрах конфигурации
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")
        
        # Извлечение признаков MFCC с помощью библиотеки librosa
        data = librosa.feature.mfcc(data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
        data = np.expand_dims(data, axis=-1)
        X[i,] = data
    return X

In [10]:
# Задаём параметры: частота дискретизации, длительность звукового файла, количество папок для кросс-валидации,
#  использование MFCC, количество признаков MFCC

config = Config(sampling_rate=44100, audio_duration=2, n_folds=10, 
                learning_rate=0.001, use_mfcc=True, n_mfcc=20)

In [12]:
X_train = prepare_data(train, config, './SVA/audio_train/')
X_test = prepare_data(test, config, './SVA/audio_test/')

0
00044347.wav
1
001ca53d.wav
2
002d256b.wav
3
0033e230.wav
4
00353774.wav
5
003b91e8.wav
6
003da8e5.wav
7
0048fd00.wav
8
004ad66f.wav
9
0063ab88.wav
10
006f2f32.wav
11
0075d39c.wav
12
00780200.wav
13
0079d310.wav
14
0091fc7f.wav
15
0097160c.wav
16
00ad7068.wav
17
00c5808a.wav
18
00c82919.wav
19
00c934d7.wav
20
00c9e799.wav
21
00cb787c.wav
22
00ce569f.wav
23
00d1fe46.wav
24
00d3bba3.wav
25
00d40fa2.wav
26
00d9fa61.wav
27
00e2b4cd.wav
28
00f88dc5.wav
29
00fbb28b.wav
30
00fcbab2.wav
31
010aa387.wav
32
011a2185.wav
33
0120d246.wav
34
01235a12.wav
35
01257aad.wav
36
01302128.wav
37
013264d3.wav
38
013c3135.wav
39
01506d76.wav
40
015cf474.wav
41
0160d55e.wav
42
01638f61.wav
43
0172a2a5.wav
44
017ea24e.wav
45
01811e48.wav
46
0184c390.wav
47
018863f5.wav
48
018a10bb.wav
49
018b1df6.wav
50
018d1dc4.wav
51
0193042e.wav
52
01974c7c.wav
53
019aae9d.wav
54
019d2a2c.wav
55
01a36643.wav
56
01a39e95.wav
57
01a59a61.wav
58
01a59c11.wav
59
01a5dc85.wav
60
01b9f44a.wav
61
01c2f88b.wav
62
01d2475c.wav
63

492
0e439e01.wav
493
0e451691.wav
494
0e4583d5.wav
495
0e51f1d0.wav
496
0e544eb8.wav
497
0e5c9ed0.wav
498
0e5cab29.wav
499
0e630318.wav
500
0e645225.wav
501
0e6e1f35.wav
502
0e7622f0.wav
503
0e7cd4eb.wav
504
0e7f9fdb.wav
505
0e89fb66.wav
506
0e974517.wav
507
0e979b8c.wav
508
0e9cb14e.wav
509
0ea3dbaf.wav
510
0eb63f5a.wav
511
0ec012fc.wav
512
0ecb56ad.wav
513
0ed06544.wav
514
0ede064f.wav
515
0ee42262.wav
516
0eeaebcb.wav
517
0ef9a602.wav
518
0f01e836.wav
519
0f18c55b.wav
520
0f1a5928.wav
521
0f1dbff5.wav
522
0f23d7e4.wav
523
0f274e7a.wav
524
0f2903db.wav
525
0f2ab678.wav
526
0f34d08e.wav
527
0f36e49a.wav
528
0f444884.wav
529
0f5286d1.wav
530
0f53ac39.wav
531
0f5c62dd.wav
532
0f6e2a63.wav
533
0f70f92f.wav
534
0f72651c.wav
535
0f7f0d86.wav
536
0f936928.wav
537
0f958ff0.wav
538
0f96c40d.wav
539
0fa3ee45.wav
540
0fa61324.wav
541
0fafb777.wav
542
0fb0c2e1.wav
543
0fb464cf.wav
544
0fc39673.wav
545
0fcdc341.wav
546
0fd52405.wav
547
0fe6c47d.wav
548
0ff26813.wav
549
0ff3a423.wav
550
0ff4262d.w

984
1b2d8997.wav
985
1b322081.wav
986
1b34065d.wav
987
1b35f18e.wav
988
1b3b537f.wav
989
1b43bbe6.wav
990
1b5d4f7c.wav
991
1b643c7f.wav
992
1b66a9f0.wav
993
1b738acd.wav
994
1b73bbfd.wav
995
1b7b6f2d.wav
996
1b813314.wav
997
1b8986df.wav
998
1b8de009.wav
999
1b8de5c7.wav
1000
1b9109be.wav
1001
1b9386ef.wav
1002
1b93aeba.wav
1003
1b93b53c.wav
1004
1b97c39a.wav
1005
1b9c5af1.wav
1006
1b9ea226.wav
1007
1bbdae38.wav
1008
1bc55c17.wav
1009
1bc8eae1.wav
1010
1bcb3c6f.wav
1011
1bcf0664.wav
1012
1bd3ba0b.wav
1013
1bda949f.wav
1014
1bdc4940.wav
1015
1bec87d9.wav
1016
1bf70cc6.wav
1017
1bf859a7.wav
1018
1c0d5682.wav
1019
1c133d0e.wav
1020
1c189121.wav
1021
1c2b5b18.wav
1022
1c3b4067.wav
1023
1c3b61c8.wav
1024
1c3d00bf.wav
1025
1c3e1a54.wav
1026
1c3ed083.wav
1027
1c4c0a51.wav
1028
1c4d9c4c.wav
1029
1c520cae.wav
1030
1c590c46.wav
1031
1c603b9f.wav
1032
1c67a5b9.wav
1033
1c6b07b8.wav
1034
1c720526.wav
1035
1c76a229.wav
1036
1c78595a.wav
1037
1c8943c0.wav
1038
1c8cddc2.wav
1039
1c9a423f.wav
1040
1ca

1450
27c53672.wav
1451
27ce7acc.wav
1452
27cf6ec2.wav
1453
27d3b396.wav
1454
27d679b4.wav
1455
27e6d2ac.wav
1456
27fbb25c.wav
1457
27ff9eea.wav
1458
280b5439.wav
1459
280f761e.wav
1460
281c7727.wav
1461
282aa031.wav
1462
282f2473.wav
1463
283671c1.wav
1464
283a585f.wav
1465
2846dc18.wav
1466
28492d31.wav
1467
284a6757.wav
1468
2857b67f.wav
1469
285a1825.wav
1470
285a7bac.wav
1471
285c42de.wav
1472
28634acc.wav
1473
286580a3.wav
1474
28712a45.wav
1475
287259da.wav
1476
287c38ed.wav
1477
28877d48.wav
1478
288821ab.wav
1479
288d0dff.wav
1480
289250ae.wav
1481
289cc6a8.wav
1482
289e5d2a.wav
1483
289e8e59.wav
1484
28a9502f.wav
1485
28aa7f31.wav
1486
28ab5413.wav
1487
28aec29b.wav
1488
28b75c35.wav
1489
28bea0af.wav
1490
28bf1f65.wav
1491
28c13a63.wav
1492
28d634e9.wav
1493
28d80cf6.wav
1494
28e0f162.wav
1495
28e413d2.wav
1496
28e78a3c.wav
1497
28ed1ac8.wav
1498
28f3b63c.wav
1499
28f5dfd6.wav
1500
2903714d.wav
1501
2904b427.wav
1502
29076655.wav
1503
290976f9.wav
1504
290d7e7d.wav
1505
29127

1923
35669b76.wav
1924
356f6f26.wav
1925
35717b0f.wav
1926
35721b6a.wav
1927
357a5b9a.wav
1928
357f7e95.wav
1929
358139e5.wav
1930
35839982.wav
1931
35844050.wav
1932
3586bc2d.wav
1933
3592e056.wav
1934
35950597.wav
1935
35a38d01.wav
1936
35accc11.wav
1937
35ad5cd8.wav
1938
35bd35da.wav
1939
35d75ab7.wav
1940
35d7ac24.wav
1941
35e0b4ea.wav
1942
35e97db3.wav
1943
35f05c5d.wav
1944
35f0e452.wav
1945
35f2639c.wav
1946
35f589b8.wav
1947
35f6b357.wav
1948
35fb9b2e.wav
1949
35fd6638.wav
1950
3600d08e.wav
1951
360174e1.wav
1952
3601d225.wav
1953
361230e1.wav
1954
36177faa.wav
1955
3618cb03.wav
1956
3623e99e.wav
1957
3626eabd.wav
1958
36315bea.wav
1959
36342286.wav
1960
363c41b3.wav
1961
3640c702.wav
1962
3649d60c.wav
1963
3650faee.wav
1964
36514399.wav
1965
365bee4a.wav
1966
365d9809.wav
1967
3662e614.wav
1968
366664bb.wav
1969
36698953.wav
1970
366a5492.wav
1971
3679b950.wav
1972
367ad7b1.wav
1973
367e990f.wav
1974
368137a8.wav
1975
368b7628.wav
1976
368d4b71.wav
1977
368de55c.wav
1978
36916

...

7366
c82178d8.wav
7367
c82443fc.wav
7368
c825d5f4.wav
7369
c831423c.wav
7370
c8582b0e.wav
7371
c865bec6.wav
7372
c87c0c04.wav
7373
c8817351.wav
7374
c887048f.wav
7375
c8adbde3.wav
7376
c8ec6e00.wav
7377
c8f167c0.wav
7378
c8f2a15e.wav
7379
c8fa5db4.wav
7380
c9051509.wav
7381
c90b811c.wav
7382
c9172469.wav
7383
c91a3134.wav
7384
c91ba6f8.wav
7385
c91bf2d3.wav
7386
c91c8c4a.wav
7387
c9220d75.wav
7388
c922f897.wav
7389
c927270b.wav
7390
c92a3475.wav
7391
c92b4c34.wav
7392
c92e687c.wav
7393
c9375679.wav
7394
c93ac13c.wav
7395
c93c1b78.wav
7396
c949e9e5.wav
7397
c94f039b.wav
7398
c95182e3.wav
7399
c954bb16.wav
7400
c959b1ef.wav
7401
c95aa254.wav
7402
c9614018.wav
7403
c967660f.wav
7404
c9684414.wav
7405
c9855f16.wav
7406
c98fcd10.wav
7407
c9912890.wav
7408
c996ca4c.wav
7409
c99849e6.wav
7410
c998d466.wav
7411
c99a185c.wav
7412
c99fbfd4.wav
7413
c9a25af4.wav
7414
c9a6616a.wav
7415
c9a9b485.wav
7416
c9aa3c62.wav
7417
c9b8d9b3.wav
7418
c9bba203.wav
7419
c9c80279.wav
7420
c9d22284.wav
7421
c9d69

7824
d4a08f8a.wav
7825
d4a84eee.wav
7826
d4a9c21e.wav
7827
d4ad4ca9.wav
7828
d4b11bd7.wav
7829
d4b15d81.wav
7830
d4bb46ea.wav
7831
d4bc266e.wav
7832
d4bf3fb3.wav
7833
d4c3c292.wav
7834
d4ceedda.wav
7835
d4cffb09.wav
7836
d4d395ee.wav
7837
d4e0f124.wav
7838
d4e15f25.wav
7839
d4f79522.wav
7840
d4f87191.wav
7841
d4fd9545.wav
7842
d5050e6f.wav
7843
d50cf0d9.wav
7844
d51c38d9.wav
7845
d5353608.wav
7846
d53b1b89.wav
7847
d53eaa18.wav
7848
d53f0766.wav
7849
d5479db2.wav
7850
d5486a44.wav
7851
d54a7a6d.wav
7852
d5517596.wav
7853
d5563166.wav
7854
d55cb268.wav
7855
d561817b.wav
7856
d56374c5.wav
7857
d5637720.wav
7858
d5654d60.wav
7859
d58a7f38.wav
7860
d58e462a.wav
7861
d590eee3.wav
7862
d593d06a.wav
7863
d593ed57.wav
7864
d59f59fc.wav
7865
d5a4304f.wav
7866
d5a90c14.wav
7867
d5ab8880.wav
7868
d5b326f6.wav
7869
d5c8c534.wav
7870
d5d5cc87.wav
7871
d5da6270.wav
7872
d5dc7885.wav
7873
d5de51f7.wav
7874
d5e04458.wav
7875
d5e4260b.wav
7876
d5ea3aac.wav
7877
d5f0276f.wav
7878
d5f037a5.wav
7879
d5f54

8281
e1b1ee02.wav
8282
e1c172fa.wav
8283
e1c78f41.wav
8284
e1ca89e3.wav
8285
e1d88aa8.wav
8286
e1eccf73.wav
8287
e1f3b477.wav
8288
e1fc13fd.wav
8289
e218f5ab.wav
8290
e21c7c53.wav
8291
e22b1cc6.wav
8292
e233206c.wav
8293
e23393b9.wav
8294
e245a3f5.wav
8295
e2529ad3.wav
8296
e25b3e5d.wav
8297
e25f0efb.wav
8298
e26962f7.wav
8299
e269a6c9.wav
8300
e269cdb6.wav
8301
e26b56a0.wav
8302
e275b9dd.wav
8303
e27a0b20.wav
8304
e291574a.wav
8305
e29e1f8b.wav
8306
e2adb002.wav
8307
e2b92cb5.wav
8308
e2b9b01a.wav
8309
e2c0033d.wav
8310
e2c7b0c7.wav
8311
e2d0b667.wav
8312
e2d30b3a.wav
8313
e2d4c145.wav
8314
e2de4941.wav
8315
e2de57a7.wav
8316
e2de89a3.wav
8317
e2e459d7.wav
8318
e2e69418.wav
8319
e2ebb0d4.wav
8320
e2f1bb44.wav
8321
e2f7c8a6.wav
8322
e3089039.wav
8323
e3177c85.wav
8324
e3181aa6.wav
8325
e31ce7b0.wav
8326
e31d75df.wav
8327
e3315711.wav
8328
e3351910.wav
8329
e339780a.wav
8330
e349a6c3.wav
8331
e35231d0.wav
8332
e353b4bf.wav
8333
e356fa95.wav
8334
e35f12f0.wav
8335
e3630482.wav
8336
e375a

8739
eee623f3.wav
8740
eef082f3.wav
8741
eef57e88.wav
8742
eefd9b00.wav
8743
ef04ff6c.wav
8744
ef052116.wav
8745
ef075b02.wav
8746
ef0c41e1.wav
8747
ef348546.wav
8748
ef3b3cc9.wav
8749
ef3cf211.wav
8750
ef4b1baf.wav
8751
ef509d82.wav
8752
ef5365c4.wav
8753
ef54ce2d.wav
8754
ef59532a.wav
8755
ef5d3318.wav
8756
ef64e9b8.wav
8757
ef663d83.wav
8758
ef66945d.wav
8759
ef67befc.wav
8760
ef6e7500.wav
8761
ef733ddf.wav
8762
ef77ece8.wav
8763
ef8c4f83.wav
8764
ef9af038.wav
8765
ef9d7bd0.wav
8766
efa3e140.wav
8767
efa4618b.wav
8768
efa9dd4c.wav
8769
efaab7e5.wav
8770
efbbc548.wav
8771
efbf661a.wav
8772
efe4ee16.wav
8773
efecc2ff.wav
8774
eff45317.wav
8775
effdc0ab.wav
8776
f0037e36.wav
8777
f005d3c0.wav
8778
f007775c.wav
8779
f007bf62.wav
8780
f00dd617.wav
8781
f010e30d.wav
8782
f0242b31.wav
8783
f024bae2.wav
8784
f0284e49.wav
8785
f0301749.wav
8786
f0363725.wav
8787
f036bdf1.wav
8788
f04b9a80.wav
8789
f04eb3d2.wav
8790
f04fff72.wav
8791
f05030a1.wav
8792
f05791ca.wav
8793
f062c9ee.wav
8794
f06bd

9196
fac25bd4.wav
9197
fac2ca19.wav
9198
fac9183b.wav
9199
facf58c7.wav
9200
fad0ea2e.wav
9201
fad93dd0.wav
9202
fae0e6e3.wav
9203
faef306c.wav
9204
faf83687.wav
9205
fafa5987.wav
9206
fb01040a.wav
9207
fb02a136.wav
9208
fb117dad.wav
9209
fb13efba.wav
9210
fb1b7eff.wav
9211
fb2807f1.wav
9212
fb2addf4.wav
9213
fb307e58.wav
9214
fb342a8a.wav
9215
fb39c458.wav
9216
fb3a4935.wav
9217
fb3ddaa0.wav
9218
fb4aed88.wav
9219
fb4b09b1.wav
9220
fb5fb6b9.wav
9221
fb5fe58c.wav
9222
fb650776.wav
9223
fb6775ce.wav
9224
fb6796ef.wav
9225
fb6f64b1.wav
9226
fb728d88.wav
9227
fb76efa5.wav
9228
fb78e8b2.wav
9229
fb84a793.wav
9230
fb8c240d.wav
9231
fb9aefc9.wav
9232
fba0d044.wav
9233
fba1ee61.wav
9234
fba9ba05.wav
9235
fbb11370.wav
9236
fbb90132.wav
9237
fbb9b3b4.wav
9238
fbc83b12.wav
9239
fbcb4c20.wav
9240
fbce9adb.wav
9241
fbd280bc.wav
9242
fbe0bc84.wav
9243
fbedd7dd.wav
9244
fbf1ed93.wav
9245
fc0ab9f4.wav
9246
fc19a9cd.wav
9247
fc1be49c.wav
9248
fc2d8019.wav
9249
fc37a395.wav
9250
fc394206.wav
9251
fc47c

In [13]:
X_train

array([[[[-5.45138367e+02],
         [-5.44282898e+02],
         [-5.48253723e+02],
         ...,
         [-6.04755310e+02],
         [-6.05829102e+02],
         [-5.99376221e+02]],

        [[ 1.46077515e+02],
         [ 1.45004044e+02],
         [ 1.38436707e+02],
         ...,
         [ 1.25009415e+02],
         [ 1.26167931e+02],
         [ 1.33063385e+02]],

        [[-6.72886963e+01],
         [-7.26910553e+01],
         [-7.52016449e+01],
         ...,
         [-5.51718674e+01],
         [-5.06269646e+01],
         [-4.87024460e+01]],

        ...,

        [[ 4.31784058e+00],
         [ 1.36715269e+00],
         [ 1.58895195e-01],
         ...,
         [-4.63688421e+00],
         [ 1.99095547e-01],
         [ 1.17123687e+00]],

        [[-3.52722073e+00],
         [-4.00278854e+00],
         [-3.09841156e+00],
         ...,
         [-1.18811045e+01],
         [-8.68222523e+00],
         [-3.28867292e+00]],

        [[ 3.46349263e+00],
         [ 3.92348230e-01],
         [

In [14]:
X_test

array([[[[-8.55863647e+02],
         [-8.55863647e+02],
         [-8.55863647e+02],
         ...,
         [-8.55863647e+02],
         [-8.55863647e+02],
         [-8.55863647e+02]],

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        ...,

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [

In [15]:
# Приводим метки классов в виде чисел

y_train = to_categorical(train.label_idx, num_classes=config.n_classes)

In [16]:
# Нормализация данных

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [17]:
PREDICTION_FOLDER = "predictions_2d_conv"
if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if os.path.exists('./logs/' + PREDICTION_FOLDER):
    shutil.rmtree('./logs/' + PREDICTION_FOLDER)

# Для кросс-валидации используется StratifiedKFold - разновдность KFold алгоритма, которая возвращает
# стратифицированные папки c данными: каждый набор в папке содержит примерно такой же процент выборок каждого целевого класса,
# что и полный набор.
skf = StratifiedKFold(n_splits=config.n_folds)
i = 0
for train_split, val_split in skf.split(X_train, train.label_idx):
    K.clear_session()
    X, y, X_val, y_val = X_train[train_split], y_train[train_split], X_train[val_split], y_train[val_split]
    # В ходе обучения сохраняем веса лучшей модели для потенциального дальнейшего использования
    checkpoint = ModelCheckpoint('best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
    tb = TensorBoard(log_dir='.logs\\' + PREDICTION_FOLDER + '\\fold_%i'%i, write_graph=True)
    callbacks_list = [checkpoint, early, tb]
    print("#"*50)
    print("Fold: ", i)
    model = get_2d_conv_model(config)
    history = model.fit(X, y, validation_data=(X_val, y_val), callbacks=callbacks_list, 
                        batch_size=64, epochs=config.max_epochs)
    model.load_weights('best_%d.h5'%i)

    # Сохраняем предсказания модели по тренировочным данным
    predictions = model.predict(X_train, batch_size=64, verbose=1)
    np.save(PREDICTION_FOLDER + "\\train_predictions_%d.npy"%i, predictions)

    # Сохраняем предсказания модели по тестовым данным
    predictions = model.predict(X_test, batch_size=64, verbose=1)
    np.save(PREDICTION_FOLDER + "\\test_predictions_%d.npy"%i, predictions)

    # Создание файла с результатами (submission)
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "\\predictions_%d.csv"%i)
    i += 1

##################################################
Fold:  0
Epoch 1/50
134/134 [==============================] - 57s 417ms/step - loss: 2.8781 - acc: 0.2489 - val_loss: 3.3407 - val_acc: 0.1603

Epoch 00001: val_loss improved from inf to 3.34068, saving model to best_0.h5
Epoch 2/50
134/134 [==============================] - 56s 418ms/step - loss: 2.1937 - acc: 0.4189 - val_loss: 3.0440 - val_acc: 0.2532

Epoch 00002: val_loss improved from 3.34068 to 3.04404, saving model to best_0.h5
Epoch 3/50
134/134 [==============================] - 55s 412ms/step - loss: 1.8680 - acc: 0.4969 - val_loss: 2.0620 - val_acc: 0.4378

Epoch 00003: val_loss improved from 3.04404 to 2.06205, saving model to best_0.h5
Epoch 4/50
134/134 [==============================] - 55s 411ms/step - loss: 1.6618 - acc: 0.5443 - val_loss: 1.9813 - val_acc: 0.4631

Epoch 00004: val_loss improved from 2.06205 to 1.98127, saving model to best_0.h5
Epoch 5/50
134/134 [==============================] - 56s 414ms/step - l

134/134 [==============================] - 55s 413ms/step - loss: 1.6892 - acc: 0.5427 - val_loss: 1.8358 - val_acc: 0.4678

Epoch 00004: val_loss improved from 1.85855 to 1.83582, saving model to best_5.h5
Epoch 5/50
134/134 [==============================] - 56s 418ms/step - loss: 1.5344 - acc: 0.5840 - val_loss: 1.9821 - val_acc: 0.4688

Epoch 00005: val_loss did not improve from 1.83582
Epoch 6/50
134/134 [==============================] - 55s 413ms/step - loss: 1.4124 - acc: 0.6113 - val_loss: 1.7244 - val_acc: 0.5312

Epoch 00006: val_loss improved from 1.83582 to 1.72441, saving model to best_5.h5
Epoch 7/50
134/134 [==============================] - 55s 413ms/step - loss: 1.3107 - acc: 0.6349 - val_loss: 1.8812 - val_acc: 0.4879

Epoch 00007: val_loss did not improve from 1.72441
Epoch 8/50
134/134 [==============================] - 55s 413ms/step - loss: 1.1946 - acc: 0.6671 - val_loss: 1.6139 - val_acc: 0.5628

Epoch 00008: val_loss improved from 1.72441 to 1.61395, saving mo

134/134 [==============================] - 55s 414ms/step - loss: 0.9172 - acc: 0.7414 - val_loss: 1.7577 - val_acc: 0.5470

Epoch 00010: val_loss did not improve from 1.65297
Epoch 11/50
134/134 [==============================] - 55s 414ms/step - loss: 0.8522 - acc: 0.7617 - val_loss: 1.6311 - val_acc: 0.5744

Epoch 00011: val_loss improved from 1.65297 to 1.63107, saving model to best_7.h5
Epoch 12/50
134/134 [==============================] - 55s 414ms/step - loss: 0.7613 - acc: 0.7868 - val_loss: 1.6391 - val_acc: 0.5502

Epoch 00012: val_loss did not improve from 1.63107
Epoch 13/50
134/134 [==============================] - 55s 414ms/step - loss: 0.6925 - acc: 0.8054 - val_loss: 1.6444 - val_acc: 0.5702

Epoch 00013: val_loss did not improve from 1.63107
Epoch 14/50
134/134 [==============================] - 55s 414ms/step - loss: 0.6266 - acc: 0.8236 - val_loss: 1.6280 - val_acc: 0.5797

Epoch 00014: val_loss improved from 1.63107 to 1.62800, saving model to best_7.h5
Epoch 15/5


Epoch 00014: val_loss did not improve from 1.58543
Epoch 15/50
134/134 [==============================] - 56s 420ms/step - loss: 0.5550 - acc: 0.8479 - val_loss: 1.7687 - val_acc: 0.5723

Epoch 00015: val_loss did not improve from 1.58543
147/147 [==============================] - 12s 84ms/step


In [18]:
# Сбор результатов из всех записанных файлов
pred_list = []
for i in range(10):
    pred_list.append(np.load("./predictions_2d_conv/test_predictions_%d.npy"%i))
prediction = np.ones_like(pred_list[0])
for pred in pred_list:
    prediction = prediction*pred
prediction = prediction**(1./len(pred_list))
# Создание файла с результатами (submission) для платформы Kaggle
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('./SVA/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("2d_conv_ensembled_submission.csv", index=False)